# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update
!apt-get install -y maven
!pip install transformers accelerate bitsandbytes auto-gptq huggingface_hub



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,313 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,658 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-up

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from transformers import pipeline

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

Primero cargamos la rúbrica para el examen, sólo se debería cambiar la parte de corrección (modificar criterios si es necesario) y modificar la tabla HTML.

In [ ]:
# Aquí pegamos el examen y la rubrica
def examenRubrica():

            return """
Eres un evaluador de código para un examen de programación. Tu tarea es analizar la implementación del estudiante y generar una evaluación detallada que incluya una calificación numérica y una retroalimentación formativa para cada método. Para la retroalimentación ten en cuenta:
1. **Uso correcto del flujo de control:**
 - No se deben utilizar sentencias "return" dentro de bucles o condiciones de control, ya que esto puede dificultar la comprensión y el mantenimiento del código.
 - Se debe asegurar que la estructura del código permita una lectura clara y un flujo de ejecución lineal.
2. **Manejo adecuado de variables locales:**
 - Evitar la creación de variables locales innecesarias.
 - Utilizar nombres descriptivos y claros para las variables que se requieren.
3. **Eficiencia y Mantenimiento:**
 - Verifica que se implementen algoritmos eficientes en términos de complejidad temporal y espacial.
 - Se debe evaluar que el código sea fácilmente mantenible y extensible en el futuro.
4. **Funcionalidad:**
 - El código debe funcionar en llamadas correctas sin errores.
 - Cuando los parámetros de entrada no estén restringidos, el código de gestionar correctamente la introducción de parámetros incorrectos (por ejemplo valores NULL, árbol vacío, etc)
**Para la calificación de cada método**. Si la funcionalidad es correcta (punto 4 anterior), tendrá un 100% de la nota del método. Puedes restar a esta puntuación hasta un:
 - 10% si "Uso correcto del flujo de control" (punto 1 anterior)
 - 5% si "Manejo adecuado de variables locales"  (punto 2 anterior)
 - 20% si no es eficiente o es complicado el mantenimiento  (punto 3 anterior)
**Formato de Salida:** La salida de la evaluación debe ser en HTML y contendrá:
 - Una línea con la calificación final sobre 10 y añade <br> al final.
 - Una tabla HTML con tres columnas: "Método", "Puntuación" y "Feedback".
 - Cada fila de la tabla debe corresponder a uno de los ítems evaluados.

Estos son los métodos a evaluar y dar una retroalimentación formativa:
 - `void insert(K key)` (1 punto): inserta una clave en el árbol. Si la clave ya existe, reemplaza el nodo existente con la nueva clave. Devuelve **EmptySearchTreeException** si `key` es null.
 - `K search(K key)` (1.5 puntos): busca la clave en el árbol y la devuelve si está presente. Si no se encuentra, devuelve null. Devuelve **EmptySearchTreeException** si `key` es null.
 - `void delete(K key)` (1 punto): Elimina una clave del árbol si está presente. Devuelve **EmptySearchTreeException** si `key` es null.
 - `void clear()` (1 punto): Elimina todos los elementos del árbol, dejándolo vacío.
 - `void deleteMinimum()` (0.5 punto): Elimina la clave más pequeña del árbol. Utiliza el método *splay* para mover el mínimo. Devuelve **EmptySearchTreeException** si el árbol es vacío.
 - `zigzigRight(Node<K> node)` (1 punto): Este método realiza una doble rotación a derecha dado el abuelo (`Node<K> node`). Tras realizar la primera rotación, si el hijo izquierdo es NULL, no realiza la segunda, devolviendo el nodo con una sola rotación aplicada. El nodo de entrada es siempre diferente de NULL.
 - `zigzigLeft(Node<K> node)` (1 punto): Este método realiza una doble rotación a izquierda dado el abuelo (`Node<K> node`). Tras realizar la primera rotación, si el hijo derecho es NULL, no realiza la segunda, devolviendo el nodo con una sola rotación aplicada.El nodo de entrada es siempre diferente de NULL.
 - `zigzagRightLeft(Node<K> node)` (1 punto): Este método realiza una doble rotación, primero a derecha y luego a izquierda dado el abuelo (`Node<K> node`). La primera rotación **no se realiza si no hay subárbol en la rama izquierda del hijo derecho.** El nodo de entrada es siempre diferente de NULL.
 - `zigzagLeftRight(Node<K> node)` (1 punto): Este método realiza una doble rotación, primero a izquierda y luego a derecha dado el abuelo (`Node<K> node`). La primera rotación **no se realiza si no hay subárbol en la rama derecha del hijo izquierdo.** El nodo de entrada es siempre diferente de NULL.
 - `copyOf(SearchTree<K> that)` (0.5 puntos): Realiza una copia de los datos.
 - `copyOf(SplayTree<K> that)` (0.5 puntos): Realiza una copia de los datos manteniendo la misma estructura.

 Los siguientes métodos ya se dan implementados y no se deben evaluar:
 private Node<K> rotateRight(Node<K> node)
 private Node<K> rotateLeft(Node<K> node)
 provate Node<K> splay(Node<K> node, K key)
 private Node<K> zigLeft(Node<K> node)
 private Node<K> zigRight(Node<K> node)
 public K minimum()

 ** Ejemplo de salida **:
La suma de las calificaciones de los métodos es: 2/10

<table border="1">
  <tr>
    <th>Método</th>
    <th>Puntuación</th>
    <th>Retroalimentación</th>
  </tr>
  <tr>
    <td>insert(K key)</td>
    <td>0</td>
    <td>**Funcionalidad:** El método presenta errores funcionales. Utiliza `search(key) == key`, lo cual compara referencias en lugar de igualdad de valores, esto impide una correcta detección de claves duplicadas. Además, la lógica de inserción dentro del bucle no maneja adecuadamente todos los casos, lo que podría llevar a inserciones incorrectas o a nodos huérfanos.
    **Flujo de Control:** Se utilizan sentencias `return` dentro de condiciones, lo que afecta la claridad del flujo y su mantenimiento.
    **Manejo de Variables:** Se crean variables innecesarias como `padre` que puede ser eliminadas.
    **Eficiencia y Mantenimiento:** La complejidad del algoritmo de inserción no es óptima (usas varias veces el método *splay* y el código es difícil de mantener debido a la lógica de control poco clara.
    **Sugerencia Formativa:** Cuando desarollas, una vez funcionan los casos base, tiene que probar os casos inusuales (un sólo nodo, árbol vacío, etc) para evitar errores inesperados en el código.</td>
  </tr>
  <tr>
    <td>search(K key)</td>
    <td>0</td>
    <td>**Funcionalidad:** El método no maneja correctamente el caso en que el árbol está vacío, lo que puede causar una excepción `NullPointerException`. Además, utiliza `search(key) == key`, lo cual es incorrecto ya que compara referencias en lugar de igualdad de valores.
    **Flujo de Control:** Evita el uso de `return` dentro del bucle. Puedes añadir la condición del if negada al bucle y te ahorras el tener un if dentro.
    **Manejo de Variables:** Las variables locales están bien nombradas.
    **Eficiencia y Mantenimiento:** La falta de manejo adecuado de excepciones afecta la eficiencia y robustez del método.
    **Sugerencia Formativa:** Como norma debes revisar que los parámetros de entrada, en todos los rangos posibles. </td>

  </tr>
  <tr>
    <td>delete(K key)</td>
    <td>0</td>
    <td>**Funcionalidad:** El método no realiza la eliminación real de la clave en el árbol. Revisa bien con el depurador y observa como no se elimina.
    **Flujo de Control:** La lógica de control es insuficiente para lograr la eliminación.
    **Manejo de Variables:** La variable `padre` no se utiliza de manera efectiva.
    **Eficiencia y Mantenimiento:** No funciona correctamente, no se puede valorar.
    **Sugerencia Formativa:** ¡Usa el depurador! Ver paso a paso como el código realiza lo que tienes en mente es muy útil para encontrar errores y asegurar que hace lo que tienes pensado.</td>

  </tr>
  <tr>
    <td>clear()</td>
    <td>1</td>
    <td>**Funcionalidad:** Implementación correcta que elimina todos los elementos del árbol.
    **Flujo de Control:** El flujo es claro y lineal.
    **Manejo de Variables:** No se utilizan variables locales innecesarias.
    **Eficiencia y Mantenimiento:** El método es eficiente y fácil de mantener.
    **Sugerencia Formativa:** Ninguna.</td>

  </tr>
  <tr>
    <td>deleteMinimum()</td>
    <td>0</td>
    <td>**Funcionalidad:** El método no maneja correctamente el caso en que el árbol está vacío, lo que debería lanzar una excepción `EmptySearchTreeException`. Además, la lógica para eliminar el mínimo puede dejar inconsistencias en la estructura del árbol (observa que cuando tiene parte derecha, borras todo)
    **Flujo de Control:** Uso inadecuado de bucles y condiciones que pueden llevar a estados inconsistentes.
    **Manejo de Variables:** Las variables son las necesarias, pero su uso no es correcto.
    **Eficiencia y Mantenimiento:** La implementación es eficiente, aunque la funcionalidad es erronea.
    **Sugerencia Formativa:** Es importante revisar los casos extremos, no sólo lo más común. Usa el depurador para forzar valores a variables que te lleven por todos los caminos base del método. </td>
  </tr>
  <tr>
    <td>zigzigRight(Node&lt;K&gt; node)</td>
    <td>0</td>
    <td>**Funcionalidad:** La implementación de la doble rotación a la derecha es incorrecta. Realiza rotaciones sobre el nieto en lugar del nodo padre, lo que puede desbalancear el árbol.
**Flujo de Control:** Las condiciones para realizar las rotaciones son inadecuadas y pueden provocar errores.
**Manejo de Variables:** Las variables `nodeToRotate` y `parent` no facilitan un seguimiento claro de las rotaciones. Rehusa la misma variable para realizar las rotaciones y devuelve esa variable, no es necesario crear variables locales intermedias.
**Eficiencia y Mantenimiento:** La lógica implementada es complicada y propensa a errores, dificultando el mantenimiento.
**Sugerencia Formativa:** Es de suma importancia leer bien los enunciados. Invierte tiempo en entender lo que se pide y pregunta si tienes dudas, este método viene bien desglosado en el enunciado de la prueba</td>

  </tr>
  <tr>
    <td>zigzigLeft(Node&lt;K&gt; node)</td>
    <td>0</td>
    <td>**Funcionalidad:** Similar a `zigzigRight`, la doble rotación a la izquierda está mal implementada, lo que puede llevar a una estructura de árbol incorrecta.
**Flujo de Control:** Las rotaciones no se aplican correctamente debido a condiciones mal definidas.
**Manejo de Variables:** Las variables utilizadas no reflejan claramente la estructura del árbol.
**Eficiencia y Mantenimiento:** La implementación es ineficiente y difícil de mantener debido a la lógica compleja y defectuosa.
**Sugerencia Formativa:** Mismo comentario que en el método `zigzigRight`</td>
  </tr>
  <tr>
    <td>zigzagRightLeft(Node&lt;K&gt; node)</td>
    <td>0</td>
    <td>**Funcionalidad:** La doble rotación en este método no maneja correctamente todos los casos posibles, especialmente cuando no existe un subárbol en la rama izquierda del hijo derecho.
**Flujo de Control:** La lógica para decidir cuándo realizar cada rotación es insuficiente.
**Manejo de Variables:** Las variables `nodeToRotate` y `parent` no están gestionadas de manera que faciliten el mantenimiento.
**Eficiencia y Mantenimiento:** La implementación propensa a errores complica la eficiencia y el mantenimiento del código.
**Sugerencia Formativa:** Revisa siempre los casos inusuales de parámetros de entrada a los métodos.</td>
  </tr>
  <tr>
    <td>zigzagLeftRight(Node&lt;K&gt; node)</td>
    <td>0</td>
    <td>**Funcionalidad:** La implementación de la doble rotación izquierda-derecha no gestiona adecuadamente todos los escenarios, especialmente cuando no hay un subárbol en la rama derecha del hijo izquierdo.
**Flujo de Control:** Las condiciones para realizar rotaciones son incompletas y pueden llevar a errores.
**Manejo de Variables:** Las variables utilizadas no permiten un seguimiento claro de las operaciones realizadas.
**Eficiencia y Mantenimiento:** La lógica compleja y defectuosa hace que el método sea difícil de mantener y poco eficiente.
**Sugerencia Formativa:** Revisa siempre los casos inusuales de parámetros de entrada a los métodos.</td>

  </tr>
  <tr>
    <td>copyOf(SearchTree&lt;K&gt; that)</td>
    <td>1</td>
    <td>**Funcionalidad:** El método realiza una copia de los datos de forma correcta..
**Flujo de Control:** El flujo es adecuado para copiar elementos.
**Manejo de Variables:** Las variables utilizadas son adecuadas.
**Eficiencia y Mantenimiento:** El método es eficiente en su copia, no se puede optimizar más.
**Sugerencia Formativa:** Ninguna.</td>
  </tr>
  <tr>
    <td>copyOf(SplayTree&lt;K&gt; that)</td>
    <td>0</td>
    <td>**Funcionalidad:** El método realiza una copia de los datos pero no mantiene la estructura original del árbol, lo que no cumple con el requisito de mantener la misma estructura.
  **Flujo de Control:** El flujo es adecuado para copiar elementos, pero no para preservar la estructura.
  **Manejo de Variables:** Las variables utilizadas son adecuadas pero insuficientes para mantener la estructura original.
  **Eficiencia y Mantenimiento:** La falta de preservación de la estructura original afecta la utilidad y mantenibilidad del método.
  **Sugerencia Formativa:** En los constructures de fábrica, cuando el parámetro de entrada tiene el mismo tipo que el objeto del método, se puede acceder a su estructura y copiarlo.</td>
  </tr>
</table>
            """

In [ ]:
# set up environment
#Iteramos y vamos copiando
import subprocess
import os


def userPrompt(userSol,compilation_output):
    return f""" ### Evalua el siguiente código del estudiante.{compilation_output}
                {userSol}"""

In [ ]:
def readExamen(input_file,MODEL):
    # Get the path of the current Python file



    dest = "/content/drive/MyDrive/Colab Notebooks/ARBOLES/EspacioTrabajo/src/main/java/org/uma/ed/datastructures/searchtree/SplayTree.java"
    cp_command = f'cp "{input_file}" {dest}'
    subprocess.run(cp_command, shell=True)
    print("Copia realizada")

                                                                       # Command to execute
    command = 'mvn compile'

    # Run the command and capture only stderr
    test_results="Compila con ERRORES. "
    try:
        result = subprocess.run(
            command, shell=True, text=True,capture_output=True
        )


        compilation_output = "Compila con ERRORES." if "COMPILATION ERROR" in result.stdout else "Compila correctamente. "


    except Exception as e:
        print("An error occurred while running the compilation command:", str(e))

    print("execute javac")

    userCode = ""
    with open(dest, "r", encoding="utf-8") as infile:
        for line_number, line_content in enumerate(infile, start=1):
            userCode += f"{line_content}"
    return userPrompt(userCode,compilation_output)[:150000]


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Definir el modelo
MODEL_NAME = "Qwen/Qwen2.5-32B-Instruct"

# Configuración para usar 4-bit Quantization con bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Cargar en 4-bit para reducir el uso de memoria
    bnb_4bit_compute_dtype=torch.float16,  # Usa float16 para mejor compatibilidad
    bnb_4bit_use_double_quant=True,  # Doble cuantización para mayor eficiencia
    bnb_4bit_quant_type="nf4"  # Cuantización NF4 para mejor precisión
)

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Cargar el modelo con la configuración de cuantización
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,  # Aplicar cuantización
    device_map="auto",  # Distribuir el modelo en la GPU disponible
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00015-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00016-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00017-of-00017.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
import re
def EvaluarExamen(estudiante, MODEL):

  # Construimos el mensaje para el modelo
  message = [
      {"role": "user", "content": examenRubrica() + readExamen(estudiante, MODEL)}
  ]

  print(message)
  # Convertir los mensajes en formato de entrada para el modelo
  prompt = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

  # Generar la respuesta con el modelo
  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Enviar a GPU si está disponible
  with torch.no_grad():
      output_ids = model.generate(**inputs, max_new_tokens=2000)

  # Decodificar la respuesta generada
  response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  #print(response)
  response = re.sub(r".*?\}\s*assistant\s*", "", response, flags=re.DOTALL)

  # Eliminar `{` si aparece al inicio
  response = response.lstrip("{")
  return response

In [ ]:
estudiante="/content/drive/MyDrive/Colab Notebooks/ARBOLES/examenes/Aguilera Gonzalez Juan Carlos_4377904_assignsubmission_file_SplayTree.java"
evaluation=EvaluarExamen(estudiante,MODEL_NAME)
print(evaluation)

Copia realizada
execute javac
[{'role': 'user', 'content': '\nEres un evaluador de código para un examen de programación. Tu tarea es analizar la implementación del estudiante y generar una evaluación detallada que incluya una calificación numérica y una retroalimentación formativa para cada método. Para la retroalimentación ten en cuenta:\n1. **Uso correcto del flujo de control:**\n - No se deben utilizar sentencias "return" dentro de bucles o condiciones de control, ya que esto puede dificultar la comprensión y el mantenimiento del código.\n - Se debe asegurar que la estructura del código permita una lectura clara y un flujo de ejecución lineal.\n2. **Manejo adecuado de variables locales:**\n - Evitar la creación de variables locales innecesarias.\n - Utilizar nombres descriptivos y claros para las variables que se requieren.\n3. **Eficiencia y Mantenimiento:**\n - Verifica que se implementen algoritmos eficientes en términos de complejidad temporal y espacial.\n - Se debe evaluar qu

In [10]:
import os
import time
# Define la carpeta donde están los archivos .java
carpeta = "/content/drive/MyDrive/Colab Notebooks/ARBOLES/examenes"
archivo_centralizado = "/content/drive/MyDrive/Colab Notebooks/ARBOLES/examenes/resultados.txt"
resultados_globales = []
# Función para procesar todos los archivos .java
def procesar_examenes(carpeta, modelo):
    # Recorre todos los archivos en la carpeta
    for filename in os.listdir(carpeta):
        # Solo procesa los archivos con extensión .java
        if filename.endswith(".java"):
            print("---------------------------")
            print("---------------------------")
            print("---------------------------")
            print(filename)
            estudiante = os.path.join(carpeta, filename)


            # Llama a la función EvaluarExamen y obtén el resultado
            resultado = EvaluarExamen(estudiante, modelo)
            print("Resultado obtenido")
            print("resultado")

            resultados_globales.append(f"{estudiante}:{resultado}")

            # Crea el nombre del archivo de salida con extensión .txt
            archivo_resultado = os.path.splitext(estudiante)[0] + ".html"

            # Guarda el resultado en el archivo .txt
            with open(archivo_resultado, "w") as f:
                f.write(resultado)
                print(f"Resultado guardado en: {archivo_resultado}")

    with open(archivo_centralizado, "w") as f:
        f.write("\n".join(resultados_globales))
        print(f"Resultados centralizados guardados en: {archivo_centralizado}")


# Llamada a la función con el modelo
procesar_examenes(carpeta, MODEL_NAME)


---------------------------
---------------------------
---------------------------
Wessels Mongelos Mathew_4377865_assignsubmission_file_SplayTree.java
Copia realizada
execute javac
[{'role': 'user', 'content': '\nEres un evaluador de código para un examen de programación. Tu tarea es analizar la implementación del estudiante y generar una evaluación detallada que incluya una calificación numérica y una retroalimentación formativa para cada método. Para la retroalimentación ten en cuenta:\n1. **Uso correcto del flujo de control:**\n - No se deben utilizar sentencias "return" dentro de bucles o condiciones de control, ya que esto puede dificultar la comprensión y el mantenimiento del código.\n - Se debe asegurar que la estructura del código permita una lectura clara y un flujo de ejecución lineal.\n2. **Manejo adecuado de variables locales:**\n - Evitar la creación de variables locales innecesarias.\n - Utilizar nombres descriptivos y claros para las variables que se requieren.\n3. **E